# DIVAnd with advection-diffusion-decay-sources

## Implementation into DIVAnd
add constraint on tracer equation via C and accept error on sources $s_o$ via last term:

Try to find field $x$ and sources $s$ to minimize

$x^T B^{-1} x+  (Hx-y_o)^TR^{-1}(Hx-y_o) + (Cx -H_s^T s)^T Q^{-1}(Cx -H_s^T s)+ (s-s_o)^T R_s^{-1} (s-s_o)$ 

Since sources unknown or with errors: minimisation on them 

$-H_s Q^{-1}(Cx -H_s^T s)+R_s^{-1} (s-s_o)=0$ 

Provides

$s=(H_s Q^{-1} H_s^T + R_s^{-1})^{-1}  [ H_s Q^{-1} Cx+ R_s^{-1}s_o]$

Putting back into function to minimize

$(Cx -H_s^T s)= [I - H_s^T (H_s Q^{-1} H_s^T + R_s^{-1})^{-1}H_s Q^{-1}   ] Cx  - H_s^T (H_s Q^{-1} H_s^T + R_s^{-1})^{-1} R_s^{-1}s_o= \tilde{C}x - {H}_s^T \tilde{s}_o$

$\tilde{s}_o=(H_s Q^{-1} H_s^T + R_s^{-1})^{-1} R_s^{-1} s_o$

$\tilde{S}=(H_s Q^{-1} H_s^T + R_s^{-1})^{-1} H_s Q^{-1} $

$\tilde{C}= [I-H_s^T \tilde{S}]  C$




$ (s-s_o) = (H_s Q^{-1} H_s^T + R_s^{-1})^{-1}  H_s Q^{-1} Cx    +  \tilde{s}_o - s_o = \tilde{S} C x - (s_o - \tilde{s}_o)$

So basically solve using virtual sources $\tilde{s}_o$ and adapted constraint $\tilde{C}$ and term on error on sources replaced by 

$\tilde{S} C x - (s_o - \tilde{s}_o)$

$H_s$ is just the obs operator for the sources points.

Once the solution $x$ found, one can calculate the estimated sources 

$s= \tilde{S} C x + \tilde{s}_o$

In [ ]:
import Pkg
Pkg.activate("../..")
Pkg.instantiate()
using DIVAnd
using Makie, CairoMakie
using LinearAlgebra
using SparseArrays
using SpecialFunctions
using Statistics

## Observations

In [ ]:
# pseudo observations at source places
x = [0.0];
y = [0.0];
testsize = 1991
so = [1.0]
#f = (sin.(x*6) .* cos.(y*6).+ cos.(x*6)) .+ randn(750) ;

## Grid and mask

In [ ]:
xi, yi =
    ndgrid(range(-6, stop = 10, length = testsize), range(-8, stop = 8, length = testsize));

# all points are valid points
mask = trues(size(xi));

# this problem has a simple cartesian metric
# pm is the inverse of the resolution along the 1st dimension
# pn is the inverse of the resolution along the 2nd dimension

pm = ones(size(xi)) / (xi[2, 1] - xi[1, 1]);
pn = ones(size(xi)) / (yi[1, 2] - yi[1, 1]);

# correlation length
len = 1.0;

# obs. error variance normalized by the background error variance
epsilon2 = 0.1;

gamma = 1

uscale = 1
u = uscale * ones(Float64, size(xi))
v = 0.0 .* ones(Float64, size(xi))
velocity = (u, v)

To create the advection diffusion constraint DO NOT USE ANY OTHER CONSTRAINT HERE.    
If expensive use iterative solver with no iterations, result is not important but `s` is needed
```julia
@time fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(x,y),so,len,epsilon2;maxit = 0,
                inversion = :pcg);
@show extrema(fi)
```

## Function to compute the constrains

In [ ]:
function DIVAconstr_advdifftracer(
    mask,
    pmn,
    xyi,
    xy,
    so,
    epsilon2advdiff,
    epsilon2source;
    velocity = ([zeros(Float64, size(mask)) for k in pmn]...,),
    diffusivity = zeros(Float64, ndims(mask)),
    decayrate = 0.0,
    moddim = zeros(ndims(mask)),
)
    # To add an advection-diffusion-decay-source equation.
    #  u*grad(f)=Div(K grad(f)) - gamma f + Sum so_i delta(x_i)
    # xy are the coordinates of the source terms 
    # so is a vector containing all local sources. They are dirac values. 
    # epsilon2advdiff: error on the constraint (TODO: check overall scaling)
    # epsilon2source: error in each individual local source term (can be a scalar or vector) (TODO: check overall scaling)
    # velocity contains the velocity components on the output grid xyi
    # Diffusivity contains the diffusion coefficient (scalar for isotropic, array for anisotropic)
    # Decayrate contains the decay rate gamma
    #
    #  Notes: units/dimensions of velocities, and diffusivities need to be consistent with the metrics pmn.
    #    so that in fine all terms have the same units [f]/[time], so you also need the decay rate have the same
    #    time units
    #  To add time-evolution, just deal with it as an additional dimension but 
    #      with unit advection velocity in that direction
    #      zero diffusivity in that direction
    #      repeated sources at the same position if the source is distributed over time

    #  For line sources in general (be it in time or space): create an array of sources WHICH COVER THE LINE
    #   ONE POINT PER OUTPUT GRID

    # As an output if this function, you get two constraints to be added to your other constraints as well 
    # as utility arrays that allow to assess a posterio source values consistent with the tracer observations

    iscyclic = moddim .> 0

    Labs = DIVAnd.len_harmonize(1.0, mask)

    s, D = DIVAnd.DIVAnd_operators(
        Val{:sparse},
        collect(mask),
        pmn,
        ([L .^ 2 for L in Labs]...,),
        iscyclic,
        [],
        Labs;
        coeff_laplacian = diffusivity, #ones(Float64,ndims(mask))
    )

    #@show size(s.D)

    RS = DIVAnd_obscovar(epsilon2source, length(so))

    # add observation constrain to cost function
    #@info "Adding observation constraint to cost function"
    obscon = DIVAnd.DIVAnd_obs(s, xyi, xy, so, RS, Matrix(undef, 0, 0))


    advc = DIVAnd_constr_advec(s, velocity)

    # Need to get the inverse volumes at sources to make the integral correct
    d = .*(pmn[:]...)

    ivol = dropdims(
        sum(
            obscon.H * oper_diag(Val{:sparse}, statevector_pack(s.sv, (d,))[:, 1]),
            dims = 2,
        );
        dims = 2,
    )

    HS = obscon.H
    C = advc.H - s.D + decayrate * I
    QS = advc.R * epsilon2advdiff

    # inverses are used here for convenience assuming QS and RS are diagonal
    # For line sources, one could introduce a correlation by using a tridiagonal matrix, easy to invers
    W1 = lu(HS * inv(QS) * HS' + inv(RS))

    sot = W1 \ (inv(RS) * so)

    ST = sparse(W1 \ Matrix(HS * inv(QS)))

    #
    STC = ST * C
    CT = C - HS' * STC

    #CT=C-HS'*(ST*C)

    # Advection diffusion constraint
    sourcec1 = DIVAnd.DIVAnd_constrain(HS' * sot .* ivol, QS, CT)
    # Initial guess on source constraint
    sourcec2 = DIVAnd.DIVAnd_constrain((so - sot) .* ivol, RS, STC)

    # Returns two constrains and tools for postprocessing   
    return sourcec1, sourcec2, STC, ivol, sot
end

## Run analysis with constrains
### Get constrains

In [ ]:
sourcec1, sourcec2, STC, ivol, sot = DIVAconstr_advdifftracer(
    mask,
    (pm, pn),
    (xi, yi),
    (x, y),
    so,
    0.000001,
    0.000000001;
    velocity = velocity,
    diffusivity = [0.25, 0.25],
    decayrate = gamma,
);

### Use the two constraints in addition to any other analysis parameters

In [ ]:
@time fi, sb = DIVAndrun(
    mask,
    (pm, pn),
    (xi, yi),
    (x, y),
    [1.0],
    len,
    epsilon2 * 100000;
    constraints = (sourcec1, sourcec2),
);
@show extrema(fi)

# Add diagnostic on actual source:
sbest = STC * statevector_pack(sb.sv, (fi,))[:, 1] ./ ivol + sot

## Plots

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Interpolated field")
hm = heatmap!(ax, xi[:, 1], yi[1, :], fi, colorrange = (0, 0.5), colormap = :bluesreds)
Colorbar(fig[1, 2], hm)
fig

In [ ]:
fi2 =
    so[1] / pi * 2 * exp.(2 * (xi .* u .+ yi .* v)) .*
    SpecialFunctions.besselk.(0, 2 * sqrt.((1 + gamma) .* (xi .^ 2 + yi .^ 2)));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Interpolated field")
hm = heatmap!(ax, xi[:, 1], yi[1, :], fi2, colorrange = (0, 0.5), colormap = :bluesreds)
Colorbar(fig[1, 2], hm)
fig

In [ ]:
for r = 0.5:0.01:1.5
    @show r,
    norm(fi2[isfinite.(fi2)] .- r .* fi[isfinite.(fi2)]) ./ norm(fi2[isfinite.(fi2)])
end